<a href="https://colab.research.google.com/github/kianpu34593/NotSoFastMD/blob/kian_parallel_attempt_1/src/parallel_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from random import sample
import numba
import copy
import time
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
plt.rcParams['figure.figsize'] = [8.0, 6.0]
#if using google colab
from google.colab import files
uploaded = files.upload() 

Saving liquid256.txt to liquid256.txt


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
%%writefile velocity_generator.py
import numpy as np
from random import sample
import numba
#Randomlly Initialize the Velocities
@numba.njit
def random_vel_generator(n=256,T_equal=100,m=1,e_scale=1): # n=number of particles
    total_k=3*T_equal*(n-1)/2 #dimensionless
    vel_per_particle=np.zeros((n,3))
    for axis in range(3):
        vel_per_particle[:,axis]=np.random.randn(256,)-0.5
    Mom_x_total=np.sum(vel_per_particle[:,0])
    Mom_y_total=np.sum(vel_per_particle[:,1])
    Mom_z_total=np.sum(vel_per_particle[:,2])
    Mom_x_avg=Mom_x_total/n
    Mom_y_avg=Mom_y_total/n
    Mom_z_avg=Mom_z_total/n
    vel_per_particle=vel_per_particle-np.array([Mom_x_avg,Mom_y_avg,Mom_z_avg]).reshape(1,3)
    k_avg_init=0.5*(1/n)*np.sum(np.sum(vel_per_particle**2,axis=1),axis=0)
    k_avg_T_eq=total_k/n
    scaling_ratio=np.sqrt(k_avg_T_eq/k_avg_init)
    vel_per_particle=vel_per_particle*scaling_ratio
    return vel_per_particle

Writing velocity_generator.py


In [41]:
%%writefile MD_sim_algorithm.py
import numpy as np
from random import sample
import numba

class SpatialDomainData:
  def __init__(self, position=np.array([]), velocity=np.array([]), acceleration=np.array([])):
    self.P = position
    self.V = velocity
    self.A = acceleration

  def __repr__(self):
    return "(P:{}, V:{}, A:{})".format(self.P, self.V, self.A)


  def concat(self, other):
    self.P = np.concatenate((self.P, other.P), 0)
    self.V = np.concatenate((self.V, other.V), 0)
    self.A = np.concatenate((self.A, other.A), 0)

#function of pbc rule2
#@numba.njit
def pbc2(separation,L):
    separation_new=np.zeros((separation.shape[0],3))   
    for i in range(separation.shape[0]):
        separation_ind=separation[i,:]
        separation_empty=np.zeros((1,3))
        for j in range(3):
            separation_axis=numba.float64(separation_ind[j])
            #separation_axis_new=np.zeros((1,))
            if separation_axis < -L/2:
                separation_axis_new=separation_axis+L
            elif separation_axis > L/2:
                separation_axis_new=separation_axis-L
            else:
                separation_axis_new=separation_axis
            separation_empty[:,j]=separation_axis_new
        separation_new[i,:]=separation_empty
    return separation_new

#@numba.njit
def pbc1(position,L):
    x_new=np.zeros((position.shape[0],3))   
    for i in range(position.shape[0]):
        position_ind=position[i,:]
        position_empty=np.zeros((1,3))
        for j in range(3):
            position_axis=numba.float64(position_ind[j])
            #separation_axis_new=np.zeros((1,))
            if position_axis < 0:
                position_axis_new=position_axis+L
            elif position_axis > L:
                position_axis_new=position_axis-L
            else:
                position_axis_new=position_axis
            position_empty[:,j]=position_axis_new
        x_new[i,:]=position_empty
    return x_new

def separate_points(infodict, my_i):
    atoms=256
    neighb_spd=None
    for i,spd in infodict.items():
      if i!=my_i:
        if (neighb_spd is None):
          neighb_spd = copy.deepcopy(spd)
        else:
          neighb_spd.concat(spd)
    #check this
    if my_i>len(infodict):#==100:
      result=0
    else:
      result=infodict[my_i]
    return result,neighb_spd

def concatDict(infodict):
  atoms=256
  wholeDict=None
  for i,spd in infodict.items():
    
      if (wholeDict is None):
        wholeDict = copy.deepcopy(spd)
      else:
        wholeDict.concat(spd)
  
  return wholeDict

# %%
#function of dimensionless LJ: U(r)=4(r^{-12}-r^{-6})
#define the function of acceleration of LJ potential with cut_off
#@numba.njit
def LJ_accel(position,neighb_x_0,r_cut,L):
    subcube_atoms=position.shape[0]
    #careful kind of confusing
    position=np.concatenate((position,neighb_x_0),0)
    num=position.shape[0]
  
    update_accel=np.zeros((subcube_atoms,3))
    dU_drcut=48*r_cut**(-13)-24*r_cut**(-7) #evaluate at r_cutoff
    #for loop is only of subcube we are interested in but we have to account for ALL distance!
    for atom in range(subcube_atoms):
        position_other=np.concatenate((position[0:atom,:],position[atom+1:num+1,:]),axis=0)
        
        position_atom=position[atom]
        separation=position_atom-position_other   
        separation_new=pbc2(separation=separation,L=L)
        r_relat=np.sqrt(np.sum(separation_new**2,axis=1))
        
        #get out the particles inside the r_cut
        accel=np.zeros((r_relat.shape[0],3))
        #get out the particles inside the r_limit
        for i, r0 in enumerate(r_relat):
            if r0 <= r_cut:
               #active_r_relat.append(r0)
               separation_active_num=separation_new[i,:]
               vector_part=separation_active_num*(1/r0)
               scalar_part=48*r0**(-13)-24*r0**(-7)-dU_drcut
               accel_num=vector_part*scalar_part
               accel[i,:]=accel_num

        update_accel[atom,:]=np.sum(accel,axis=0)
        
    return update_accel.reshape(subcube_atoms,3)


#Function for avg KE (the system) 
#@numba.njit
def Kin_Eng(m,x_dot):
    num=x_dot.shape[0]
    Kinetic_per=0.5*m*np.sum(x_dot**2,axis=1)
    Kinetic_sum=np.sum(Kinetic_per,axis=0)
    return Kinetic_sum

#Function for avg LJ potential with cut_off
#@numba.jit(nopython=True)
def LJ_potent_nondimen(position,r_cut=2.5,L=6.8):
    num=position.shape[0]
    update_LJ=np.zeros((num-1,1))
    
    #fix value for a certain r_limit
    dU_drcut=24*r_cut**(-7)-48*r_cut**(-13)
    U_rcut=4*(r_cut**(-12)-r_cut**(-6))
    #print(numba.typeof(r_cut))
    #L_half=np.array(L/2)
    for ind, atom in enumerate(range(num-1)):
        position_relevent=position[atom:,:]
        position_other=position_relevent[1:,:]
        
        #pbc rule2
        separation=position_relevent[0,:]-position_other
        separation_new=pbc2(separation=separation,L=L)
        
        r_relat=np.sqrt(np.sum(separation_new**2,axis=1)).reshape(separation_new.shape[0],)
        LJ=[]
        #get out the particles inside the r_limit
        for i, r0 in enumerate(r_relat):
            if r0 <= r_cut:
               #active_r_relat.append(r0)
               LJ_num=4*r0**(-12)-4*r0**(-6)-U_rcut-(r0-r_cut)*dU_drcut
               LJ.append(LJ_num)
        #active_particle=r_relat<=r_cut_mat
        #active_r_relat=r_relat[active_particle]
        #active_amount=np.array(active_r_relat).shape[0]
        #if active_amount!=0:
            #LJ=4*active_r_relat**(-12)-4*active_r_relat**(-6)-np.array(U_rcut)-(active_r_relat-r_cut)*np.array(dU_drcut) #double counting issue here
            update_LJ[atom,:]=np.sum(np.array(LJ),axis=0)    
    return np.sum(update_LJ)

# %%
#Function of insta Pressue
#@numba.njit
def insta_pressure(L,T,N,position,r_cut,e_scale):
    num=position.shape[0]
    k_B=1.38064852*10**(-23)
    #k_B=1 #if dimensionless
    V=L**3
  
    pres_ideal=N*T*(k_B/e_scale)/V
    dU_drcut=24*r_cut**(-7)-48*r_cut**(-13)
    pres_virial=np.zeros((num-1,1))
    for atom in range(num-1):
        position_relevent=position[atom:,:]
        position_other=position_relevent[1:,:]
        position_atom=position_relevent[0,:]
        
        #pbc rule 2
        separation=position_atom-position_other
        separation_new=pbc2(separation=separation,L=L)
        
        r_relat=np.sqrt(np.sum(separation_new**2,axis=1)).reshape(separation_new.shape[0],)
        force=[]
        active_r_relat=[]
        #get out the particles inside the r_limit
        for i, r0 in enumerate(r_relat):
            if r0 <= r_cut:
               #active_r_relat.append(r0)
               force_num=-(24*r0**(-7)-48*r0**(-13))+dU_drcut
               force.append(force_num)
               active_r_relat.append(r0)    
               
        # #get out the particles inside the r_cut
        active_amount=np.array(active_r_relat).shape[0]
        
        rijFij=np.array(active_r_relat).reshape(1,active_amount)@np.array(force).reshape(active_amount,1)
        pres_virial[atom,:]=rijFij
    pres_insta=pres_ideal+np.sum(pres_virial,axis=0)/(3*V) #Double Count?
    return pres_insta

# %%
#Update rule: velocity Verlet
#@numba.njit
def vel_Ver(infodict,procid,dt,r_limit=2.5,L=6.8):
    ## Get the data for the current spatial domain 
      my_spd, neighs_spd= separate_points(infodict, procid) 

    ##acceleration at t0
      my_spd.A=LJ_accel(position=my_spd.P,neighb_x_0=neighs_spd.P,r_cut=r_limit,L=L) 
      
    ##velocity at dt/2
      my_spd.V=my_spd.V+my_spd.A*(dt/2)
      
    ##position at dt
      my_spd.P=my_spd.P+my_spd.V*dt
      
    ##PBC rule 1
      my_spd.P=pbc1(position=my_spd.P,L=L)
      infodict[procid]=my_spd
    ##???? Why there were 2 for-loop?
      return infodict 

def celltodic(positions,nx,ny,nz,L):
  xinterval=L/nx
  yinterval=L/ny
  zinterval=L/nz
  cell_lists={}
  for i in range (1,nx*ny*nz+1): 
    cell_lists[i]= np.array([]) 
  
  for i in range(0, positions.shape[0]):
    atom=positions[i,0:9]
    #check extra one!!!
    #if statements !!!!!!!
    #check later
    atomID=(np.floor(atom[0]/xinterval)+1+(np.floor(atom[1]/yinterval))*ny)+(np.floor(atom[2]/zinterval))*(nx*ny)
    #atomID=(np.floor(atom[0]/xinterval)+1+(np.floor(atom[1]/yinterval))+1)+(np.floor(atom[2]/zinterval))*(nx*ny)
    atomID=int(atomID)
    assert(atomID<9)
    if atomID>9:
      print('atomID>9')
    assert(atomID>0)
    if atomID<0:
      print('atomID<0')
    cell_lists[atomID]=np.append(cell_lists[atomID],atom)
    #print(atomID)

  for i in range(1,nx*ny*nz+1):
    # I think this works but we should think more about what an "empty" shape means
    # currently when a key points to an empty cube the cube has a position vector of dimensions [0,3]
    if cell_lists[i].shape[0]!=0:
      #check shape!
      temp_reshaped=np.reshape(cell_lists[i],((cell_lists[i].shape[0])//9,9))
      temp_position = temp_reshaped[:,0:3]
      assert(temp_position.shape[1] == 3)
      temp_velocity = temp_reshaped[:,3:6]
      temp_acceleration = temp_reshaped[:,6:9]
      #some object oriented programming
      spatial_domain_data = SpatialDomainData(temp_position,
                                              temp_velocity,
                                              temp_acceleration)
      cell_lists[i] = spatial_domain_data
    else:
      cell_lists[i] = SpatialDomainData(np.empty((0,3)),
                                        np.empty((0,3)),
                                        np.empty((0,3)))

  return cell_lists

Overwriting MD_sim_algorithm.py


In [56]:
%%writefile MD_parallel_simulation.py
#The LJ potential MD simulation function
#@numba.njit
import sys
nb_path = '/content/drive/MyDrive/Colab Notebooks'
sys.path.insert(0,nb_path)
from mpi4py import MPI
import numpy as np
import time
from random import sample
import numba
import tqdm
from velocity_generator import random_vel_generator
from MD_sim_algorithm import LJ_potent_nondimen, Kin_Eng, insta_pressure, celltodic, vel_Ver
print('finished_loading')
def LJ_MD(subdiv,position_init,dt,stop_step,accel_init,r_cut,L,T_eq,e_scale,sig):
    comm = MPI.COMM_WORLD
    my_rank = comm.Get_rank()
    p = comm.Get_size()
    print('finished finding the location')
    ###Master Nodes 
    ## Reposbility: 1) assigning random velocity,
    ##              2) gather position of the atoms at each iteration to calculate KE, PE, T and P
    ##              3) redistribute the position to each cores
    if my_rank == 0:
        print(my_rank)
      #useful constant
        k_B=1.38064852*10**(-23)
        size_sim=position_init.shape[0]
    
      #velocity initialization 
        x_dot_init=random_vel_generator(n=size_sim,T_equal=T_eq,m=1,e_scale=e_scale)
      
      #initialize the info, PE, KE, T_insta, P_insta
        info=np.zeros((stop_step+1,size_sim,9))
        PE=np.zeros((stop_step+1,1))
        KE=np.zeros((stop_step+1,1))
        T_insta=np.zeros((stop_step+1,1))
        P_insta=np.zeros((stop_step+1,1))

      #start to generate value for the 0th iteration 
        info[0,:,:]=np.concatenate((position_init,x_dot_init,accel_init),axis=1) #do we need to care about the rank?
        PE[0,:]=LJ_potent_nondimen(info[0,:,0:3],r_cut=r_cut,L=L) #unitless
        KE[0,:]=Kin_Eng(m=1,x_dot=info[0,:,3:6]) #unitless
        T_insta[0,:]=2*KE[0,:]*e_scale/(3*(size_sim-1)*k_B) #k
        P_insta[0,:]=insta_pressure(L=L,T=T_insta[0],N=size_sim,position=info[0,:,0:3],r_cut=r_cut,e_scale=e_scale) #unitless
        print('finished initializing')
        ## this infotodic currently contains 8 keys. (so our mpirun has to be 9, how to get it dynamic?)
        
        for step in range(stop_step):
          #divide work
            infotodic=celltodic((info[step,:,:]),subdiv[0],subdiv[1],subdiv[2],L)
          #broadcast this dict to all processors
            #infotodic=comm.bcast(infotodic, root=0)
          #this is a little bit stupid but I dont know how to get broadcast to work.
          ### the following comment area may be better? #use scatter 
          # #distribute work  
          #   for index, item in infotodic.items():
          #       comm.send(item,dest=index) #this is slow because we will send message one by one how to do it more efficiently?
          
          #receive work from processors
            info_per_cube={} #a better way to do this?
            for procid in range(1,p):
                print('preparing to send')
                #comm.Barrier()
                comm.send(infotodic,dest=procid)
                print(my_rank,'sent')
                info_per_cube[procid]=comm.recv(source=procid) #recieving information from each processors
                print(my_rank,'received from',procid)
            info[step+1,:,:] = np.array(list(info_per_cub.values())).reshape(1,size_sim,9)
          #calculate PE KE T_insta and P_insta
            PE[step+1,:]=LJ_potent_nondimen(info[step+1,:,0:3],r_cut=r_cut,L=L)#unitless
            KE[step+1,:]=Kin_Eng(m=1,x_dot=info[step+1,:,3:6])#unitless
            T_insta[step+1,:]=2*KE[step+1,:]*e_scale/(3*(size_sim-1)*k_B)#k
            P_insta[step+1,:]=insta_pressure(L=L,T=T_insta[step+1],N=size_sim,position=info[step+1,:,0:3],r_cut=r_cut,e_scale=e_scale)#unitless
        return info,PE,KE,T_insta,P_insta  
    else:
      #receive message from bcast

        print(my_rank)
        print('preparing to receive')
        #comm.Barrier()
        infotodic=comm.recv(source=0)
        print(my_rank,'received')
        info_temp=vel_Ver(infodict=infotodic,procid=my_rank,dt=dt,r_limit=r_cut,L=L)
        comm.send(info_temp[my_rank],dest=0) 
        print(my_rank,'sent')

def saving_NVE(T_insta,P_insta,KE,PE,info,path=''):
    #save the last 1000 step xyz file (position)
    file_name=path+'position.xyz'
    comment = 'This is the position of the system during the last 1000 steps'
    with open(file_name, 'w') as f:
        f.write("%s\n" % str(info.shape[1]))
        f.write("%s\n" % comment)
        for atom in info[-1000:,:,0:3]:
            for position in atom:
                f.write("LJ ")
                f.write("%s\n" % str(position[:])[1:-2])
    #save everything else
    file_name=path+'results.txt'
    with open(file_name, 'w') as f:
        f.write('KE'+"\t"+'PE'+"\t"+"T_insta(K)"+"\t"+"P_insta"+"\n")
        for i,T in enumerate(T_insta):
            f.write(str(KE[i,:])[1:-2]+'\t'+str(PE[i,:])[1:-2]+"\t"+str(T)[1:-2]+"\t"+str(P_insta[i,:])[1:-2])
            f.write('\n')



if __name__ == "__main__":
  k_B=1.38064852*10**(-23)
  #read position file
  file_name='liquid256.txt'
  position_init=np.loadtxt(file_name)
  #specify cubes
  subdiv=np.array([2,2,2])
  #specify dt
  dt=0.002
  #specify stop_step
  stop_step=20
  #specify accel_init
  accel_init = np.zeros((position_init.shape[0],3))
  #specify r_cut
  r_cut=2.5
  #specify L
  L=6.8
  #specify e_scale
  e_scale = 1.66*10**(-21)
  #specify sigma
  sig=3.4
  #specify T_eq
  T_dimensional_equal=123
  T_eq=T_dimensional_equal*k_B/e_scale
  #call the MD_function
  start_time=time.time()
  print('calling')
  info,PE,KE,T_insta,P_insta=LJ_MD(subdiv,
                                  position_init,
                                  dt,
                                  stop_step,
                                  accel_init,
                                  r_cut,
                                  L,
                                  T_eq,
                                  e_scale,
                                  sig)
  end_time=time.time()
  print('Time taken:', int(end_time,start_time))
  saving_NVE(T_insta,P_insta,KE,PE,info)

Overwriting MD_parallel_simulation.py


In [ ]:
!mpirun --allow-run-as-root -np 9 python MD_parallel_simulation.py

finished_loading
finished_loading
finished_loading
calling
calling
finished finding the location
6
preparing to receive
calling
finished finding the location
2
preparing to receive
finished_loading
finished_loading
finished finding the location
8
preparing to receive
calling
calling
finished finding the location
3
preparing to receive
finished finding the location
0
finished_loading
calling
finished finding the location
4
preparing to receive
finished_loading
finished_loading
finished_loading
calling
finished finding the location
7
preparing to receive
calling
finished finding the location
5
preparing to receive
calling
finished finding the location
1
preparing to receive


In [19]:
path=''
path+'info.txt'

'info.txt'

In [24]:
info=np.random.rand(9,5,3)

In [25]:
for i in info:
  print(i)

[[0.67188078 0.44242771 0.67562162]
 [0.25197418 0.28178    0.00640083]
 [0.17953964 0.04367291 0.01620384]
 [0.22797102 0.81932098 0.67418992]
 [0.18500425 0.5736284  0.06770863]]
[[0.8378475  0.60806688 0.45036725]
 [0.01052179 0.79141131 0.83301951]
 [0.59394891 0.97249402 0.03545833]
 [0.26174397 0.39341321 0.66434665]
 [0.7499728  0.96756242 0.31730333]]
[[0.52962145 0.84257323 0.03791137]
 [0.71012184 0.02665231 0.93025146]
 [0.18669956 0.5968499  0.77365144]
 [0.59550404 0.41471677 0.79738249]
 [0.24203885 0.44230083 0.0033755 ]]
[[0.05389326 0.73104985 0.956448  ]
 [0.95378349 0.56360241 0.33202853]
 [0.29415761 0.40808656 0.4083257 ]
 [0.24473047 0.79736096 0.52335965]
 [0.19568035 0.74141544 0.60511017]]
[[0.07677045 0.08606742 0.17943435]
 [0.35444906 0.55926564 0.68376782]
 [0.88770079 0.99181115 0.63499278]
 [0.31232944 0.02964499 0.88209909]
 [0.5730626  0.54046318 0.06526612]]
[[0.71455112 0.94534479 0.84430299]
 [0.7363669  0.93703674 0.19752565]
 [0.66157175 0.63926989